Messdaten im table - Format

In [20]:
import pandas as pd
import numpy as np

# Daten importieren
runderstabein = pd.read_excel(r"tables/runderstabeinseitig.xlsx")       ;lrund = 0.59
eckigerstabein = pd.read_excel(r"tables/eckigerstabeinseitig.xlsx")     ;leck = 0.602
runderstabbeid = pd.read_excel(r"tables/runderstabbeidseitig.xlsx")     
eckigerstabbeid = pd.read_excel(r"tables/eckigerstabbeidseitig.xlsx")   

################################################
# Tabellen ergänzen durch D(x) und Approximation
################################################

# Für einseitige Einspannung
def func1(l,x):
    rtn = (l * x**2 - x**3 / 3) * 10**(3)
    return rtn

# Für beidseitige Einspannung
def func1(l,x):
    rtn = (3*l**2 * x - 4 * x**3) * 10**(3)
    return rtn

################################
# Runder Stab einseitig
arr1 = pd.DataFrame(runderstabein).to_numpy()
dx1 = arr1[:,2] - arr1[:,1]
approx1 = func1(lrund, arr1[:,0] * 10**(-3))
arr1 = np.c_[arr1, dx1, approx1]

# Eckiger Stab einseitig
arr2 = pd.DataFrame(eckigerstabein).to_numpy()
dx2 = arr2[:,2] - arr2[:,1]
approx2 = func1(leck, arr2[:,0] * 10**(-3))
arr2 = np.c_[arr2, dx2, approx2]

################################
# Runder Stab beidseitig
arr3 = pd.DataFrame(runderstabbeid).to_numpy()
dx3 = arr3[:,2] - arr3[:,1]
approx3 = func1(lrund, arr3[:,0] * 10**(-3))
arr3 = np.c_[arr3, dx3, approx3]

# Eckiger Stab beidseitig
arr4 = pd.DataFrame(eckigerstabbeid).to_numpy()
dx4 = arr4[:,2] - arr4[:,1]
approx4 = func1(leck, arr4[:,0] * 10**(-3))
arr4 = np.c_[arr4, dx4, approx4]

################################
# Tabellen umwandeln und printen
################################

# Tabellen umwandeln zu Excel (für print to Latex)
arr1 =  pd.DataFrame(data=arr1)
arr2 =  pd.DataFrame(data=arr2)
arr3 =  pd.DataFrame(data=arr3)
arr4 =  pd.DataFrame(data=arr4)

# Tabellen erstellen
#print(arr1.to_latex(index = False, column_format= "c c c c c", na_rep = "-", float_format="%.3f"))
#print(arr2.to_latex(index = False, column_format= "c c c c c", na_rep = "-", float_format="%.3f"))
#print(arr3.to_latex(index = False, column_format= "c c c c c", na_rep = "-", float_format="%.3f"))
#print(arr4.to_latex(index = False, column_format= "c c c c c", na_rep = "-", float_format="%.3f"))